In [1]:
from numpy import load
import numpy as np
import pandas as pd
import itertools
import requests
from io import StringIO
import pyarrow.parquet as pq
from urllib.error import HTTPError
import warnings

In [2]:
#data = load('classes_sherlock.npy', allow_pickle=True)
#print(data)

In [15]:
acceptable_columns = ['address', 'age', 'area', 'birth Date', 'birth Place', 'brand', 'city', 'continent', 'country', 'county', 'currency', 'day', 'duration', 'industry', 'language', 'location', 'manufacturer', 'name', 'nationality', 'order', 'person', 'product', 'range', 'rank', 'region','sales', 'sex', 'state', 'status', 'symbol', 'type', 'year']
acceptable_columns = np.array(acceptable_columns)
# np.save("classes_sherlock.npy", acceptable_columns, allow_pickle=False)


In [16]:
# main dataframes to add onto existing dataframe
new_train = pd.DataFrame(columns = ['type', 'values'])
new_test = pd.DataFrame(columns = ['type', 'values'])
new_val = pd.DataFrame(columns = ['type', 'values'])

# list of urls of csvs
urls = ['https://drive.google.com/file/d/1JPdeXZ5h6zM_esbpUwlpVD-hFx7ifVmm/view?usp=sharing', 'https://drive.google.com/file/d/10eD7XzIql6LOVJlW9C7Wzc1DVnECigBw/view?usp=sharing',
        'https://drive.google.com/file/d/1IfQ5SAPgdX8R84qvWxAIy-XoMGC4EuNW/view?usp=sharing','https://drive.google.com/file/d/1bHYZGXgYP7PYWrNFvpXGJXKJNgZxhyBU/view?usp=sharing']
# list of types
types = ['guuid','guuid','tax_id','social_security']

In [17]:
#loop through all urls
for i in range(len(urls)):
    url = urls[i]
    #check that url is valid
    try:
        file_id = url.split('/')[-2]
        dwn_url='https://drive.google.com/uc?id=' + file_id
        #check that data is csv
        try:
            data = pd.read_csv(dwn_url, sep=";", header=None)
            t = types[i]
            #check that data is single column
            if data.shape[1] != 1:
                print('Data is not a single column')
                continue
        except UnicodeDecodeError:
            print('Data not in csv format')
            continue
    except HTTPError:
        print('Error: Link '+ url + ' not valid')
        continue
    
    # split 0.8 to training, 0.1 to test, and 0.1 to validation
    train_idx = round(len(data)*0.8)
    test_idx = round(len(data)*0.15)
    train_data = data.iloc[:train_idx]
    test_data = data.iloc[train_idx: (train_idx+test_idx)]
    val_data = data.iloc[(train_idx+test_idx):]

    # splitting into chunks of ~five data points
    train_data = train_data.values.tolist()
    #split data from list
    train_data = list(itertools.chain(*train_data))
    splitval = train_idx/5
    train_data = [str(i) for i in train_data]
    #split into 5 seperate arrays
    train_data_split = np.array_split(train_data, splitval)
    #gather arrays into dataframe
    train_df = pd.Series(train_data_split, name='values').to_frame()
    #set train_df['type'] to current type
    train_df['type'] = t
    #convert all train_df['values'] to string
    train_df = train_df.iloc[:,[1,0]]
    train_df['values'] = [str(i) for i in train_df['values']]

    #repeat above for test_data
    splitval = round(test_idx/5)
    test_data = test_data.values.tolist()
    test_data = list(itertools.chain(*test_data))
    test_data = [str(i) for i in test_data]
    test_data_split = np.array_split(test_data, splitval)
    test_df = pd.Series(test_data_split, name='values').to_frame()
    test_df['type'] = t
    test_df = test_df.iloc[:,[1,0]]
    test_df['values'] = [str(i) for i in test_df['values']]

    #repeat above for validation_data
    splitval = round(len(val_data)/5)
    val_data = val_data.values.tolist()
    val_data = list(itertools.chain(*val_data))
    val_data = [str(i) for i in val_data]
    val_data_split = np.array_split(val_data, splitval)
    val_df = pd.Series(val_data_split, name='values').to_frame()
    val_df['type'] = t
    val_df = val_df.iloc[:,[1,0]] 
    val_df['values'] = [str(i) for i in val_df['values']]
    
    #remove warnings for append dataframe
    warnings.filterwarnings('ignore')
    
    # add to main dataframes
    new_train = new_train.append(train_df, ignore_index=True)
    new_test = new_test.append(test_df, ignore_index=True)
    new_val = new_val.append(val_df, ignore_index=True)

In [18]:
#read current train values and labels
train_df_values = pq.read_table(source="./data/raw/train_values.parquet").to_pandas()
train_df_labels = pq.read_table(source="./data/raw/train_labels.parquet").to_pandas()
#create new dataframe of current values and labels
train_df_full = train_df_labels.merge(train_df_values, left_index=True, right_index=True)
train_df_full = train_df_full[train_df_full['type'].isin(acceptable_columns)]

In [19]:
#set up old dataframe format
old_train = pd.DataFrame(columns = ['type', 'values'])
old_test = pd.DataFrame(columns = ['type', 'values'])
old_val = pd.DataFrame(columns = ['type', 'values'])

In [20]:
# split each semantic type in original Sherlock data into 0.8 for training, 0.15 for testing, 0.05 for validation
for i in range(len(acceptable_columns)):
    temp_df = train_df_full[train_df_full["type"] == acceptable_columns[i]]
    training_idx = round(len(temp_df)*0.8)
    testing_idx = round(len(temp_df)*0.15)
    training_data = temp_df.iloc[:training_idx]
    testing_data = temp_df.iloc[training_idx: (training_idx+testing_idx)]
    valid_data = temp_df.iloc[(training_idx+testing_idx):]
    
    old_train = old_train.append(training_data, ignore_index=True)
    old_test = old_test.append(testing_data, ignore_index=True)
    old_val = old_val.append(valid_data, ignore_index=True)

In [21]:
#append old test data to new test data
test_df_full = old_test.append(new_test, ignore_index=True)
test_df_values = pd.DataFrame(test_df_full, columns=['values'])
test_df_labels = pd.DataFrame(test_df_full, columns=['type'])

In [22]:
#append old train data to new train data
train_df_full = old_train.append(new_train, ignore_index=True)
train_df_values = pd.DataFrame(train_df_full, columns=['values'])
train_df_labels = pd.DataFrame(train_df_full, columns=['type'])

In [23]:
#append old validation data to new validation data
val_df_full = old_val.append(new_val, ignore_index=True)
val_df_values = pd.DataFrame(val_df_full, columns=['values'])
val_df_labels = pd.DataFrame(val_df_full, columns=['type'])

In [24]:
#save test,train,and validation labels to data folder
test_df_labels.to_parquet("./data/raw/test_labels.parquet", engine='fastparquet')
test_df_values.to_parquet("./data/raw/test_values.parquet", engine='fastparquet')

train_df_labels.to_parquet("./data/raw/train_labels.parquet", engine='fastparquet')
train_df_values.to_parquet("./data/raw/train_values.parquet", engine='fastparquet')

val_df_labels.to_parquet("./data/raw/val_labels.parquet", engine='fastparquet')
val_df_values.to_parquet("./data/raw/val_values.parquet", engine='fastparquet')



In [ ]:
# TO DO:
# add a try catch for putting in an unvalid link ~
# also maybe a try catch for if the data is not in the right format (i.e. not in a csv with a single column) ~
# get rid of any unnecessary code  
# hide warnings so the warning about append won't show up or change the code to use concat instead
# add any extra comments to make the code more readable